In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
#Tools Conv3D, MaxPool3D, relu, flatten
# 3, frames, size, size = C T H W
#possible changes = use pool1 more often instead of pool2
#add other layers

class Net(nn.Module):
    def __init__(self, num_words=340, frames=60,size=256):
        super().__init__()
        self.conv1 = nn.Conv3d(3, 32, 3, padding = 'same')
        self.conv2 = nn.Conv3d(32, 32, 3, padding = 'same')
        self.conv3 = nn.Conv3d(32, 32, 3, padding = 'same')
        self.conv4 = nn.Conv3d(32, 32, 3, padding = 'same')
        self.conv5 = nn.Conv3d(32, 64, 3, padding = 'same') 

        self.conv6 = nn.Conv3d(64, 64, 3, padding = 'same')
        self.conv7 = nn.Conv3d(64, 64, 3, padding = 'same')

        self.conv8 = nn.Conv3d(64, 128, 3, padding = 'same')
        self.conv9 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv10 = nn.Conv3d(128, 128, 3, padding = 'same')

        self.conv11 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv12 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv13 = nn.Conv3d(128, 128, 3, padding = 'same') # could go to 256/512 somewhere here
        self.conv14 = nn.Conv3d(128, 128, 3, padding = 'same')
        self.conv15 = nn.Conv3d(128, 256, 3, padding = 'same')
        self.conv16 = nn.Conv3d(256, 256, 3, padding = 'same')
        self.conv17 = nn.Conv3d(256, 256, 3, padding = 'same')
        self.conv18 = nn.Conv3d(256, 256, 3, padding = 'same')
        self.conv19 = nn.Conv3d(256, 256, 3, padding = 'same')
        self.conv20 = nn.Conv3d(256, 256, 3, padding = 'same')
        self.conv21 = nn.Conv3d(256, 256, 3, padding = 'same')
        self.conv22 = nn.Conv3d(256, 256, 3, padding = 'same')

        self.pool1 = nn.MaxPool3d((1, 2, 2))
        self.pool2 = nn.MaxPool3d(2)
        self.relu = nn.ReLU()
        self.preds1 = nn.Linear(int(256*(frames/2)*(size/64)*(size/64)), num_words)

    def forward(self, x):
        #C T H W = 3 self.frames self.size self.size
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        tmp = x
        x = self.relu(self.conv3(x))
        x = self.relu(self.conv4(x))
        x = tmp + x
        x = self.pool1(self.relu(self.conv5(x))) # 64 self.frames self.size/2 self.size/2
        tmp = x
        x = self.relu(self.conv6(x))
        x = self.relu(self.conv7(x))
        x = tmp + x 
        x = self.relu(self.conv8(x))
        tmp = x 
        x = self.relu(self.conv9(x))
        x = self.relu(self.conv10(x)) 
        x = tmp + x 
        x = self.pool1(x) # C T H W = 128 self.frames self.size/4 self.size/4
        tmp = x
        x = self.relu(self.conv11(x))
        x = self.relu(self.conv12(x))
        x = tmp + x
        x = self.pool1(x) # 128 self.frames self.size/8 self.size/8
        tmp = x
        x = self.relu(self.conv13(x)) 
        x = self.relu(self.conv14(x)) 
        x = tmp + x
        x = self.pool1(self.relu(self.conv15(x))) #256 self.frames self.size/16 self.size/16
        tmp = x
        x = self.relu(self.conv16(x)) 
        x = self.relu(self.conv17(x)) 
        x = tmp + x 
        x = self.pool1(x) # 256 self.frames self.size/32 self.size/32
        tmp = x
        x = self.relu(self.conv18(x)) 
        x = self.relu(self.conv19(x))
        x = tmp + x
        x = self.pool2(x) # 256 sel.frames/2 self.size/64 self.size/64
        tmp = x
        x = self.relu(self.conv20(x)) 
        x = self.relu(self.conv21(x))
        x = tmp + x
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.preds1(x))
        return x



In [4]:
x = torch.rand(1, 3, 60, 256, 256)

In [3]:
model = Net()

In [5]:
model(x)

tensor([[2.9595e-02, 2.0553e-02, 1.9745e-02, 4.6181e-02, 0.0000e+00, 1.8215e-02,
         1.0659e-02, 2.6596e-02, 2.7734e-03, 0.0000e+00, 1.0990e-02, 2.8617e-02,
         0.0000e+00, 1.3510e-03, 0.0000e+00, 0.0000e+00, 5.6103e-02, 9.4475e-03,
         2.8532e-02, 0.0000e+00, 2.6057e-02, 0.0000e+00, 1.3344e-02, 0.0000e+00,
         6.8253e-03, 5.2664e-03, 0.0000e+00, 4.9352e-03, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 1.5034e-04, 2.3408e-02, 0.0000e+00,
         5.9795e-02, 0.0000e+00, 1.8474e-02, 9.9461e-03, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 2.8883e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         2.4072e-02, 4.3061e-03, 0.0000e+00, 1.8086e-02, 0.0000e+00, 0.0000e+00,
         5.3076e-04, 1.8341e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 2.9735e-02,
         1.0951e-02, 0.0000e+00, 1.1297e-03, 0.0000e+00, 2.6014e-03, 1.2794e-02,
         1.3548e-02, 0.0000e+00, 3.7915e-03, 0.0000e+00, 0.0000e+00, 1.7471e-02,
         1.2329e-02, 7.3701e

In [6]:
#Parameters
from torchsummary import summary
summary(model, (3, 60, 256, 256))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1     [-1, 32, 60, 256, 256]           2,624
              ReLU-2     [-1, 32, 60, 256, 256]               0
            Conv3d-3     [-1, 32, 60, 256, 256]          27,680
              ReLU-4     [-1, 32, 60, 256, 256]               0
            Conv3d-5     [-1, 32, 60, 256, 256]          27,680
              ReLU-6     [-1, 32, 60, 256, 256]               0
            Conv3d-7     [-1, 32, 60, 256, 256]          27,680
              ReLU-8     [-1, 32, 60, 256, 256]               0
            Conv3d-9     [-1, 64, 60, 256, 256]          55,360
             ReLU-10     [-1, 64, 60, 256, 256]               0
        MaxPool3d-11     [-1, 64, 60, 128, 128]               0
           Conv3d-12     [-1, 64, 60, 128, 128]         110,656
             ReLU-13     [-1, 64, 60, 128, 128]               0
           Conv3d-14     [-1, 64, 60, 1

C:\Users\khota\miniconda3\envs\test\lib\site-packages\torchsummary\torchsummary.py:93: RuntimeWarning: overflow encountered in long_scalars
  total_output += np.prod(summary[layer]["output_shape"])


In [ ]:
#Training Loop Below

In [7]:
import torch.optim as optim
import torch.nn as nn

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
for epoch in range(50):  # loop over the dataset multiple times
    for data in trainloader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')

NameError: name 'trainloader' is not defined